In [30]:
import numpy as np

# Метод сеток для уравнения теплопроводности

## Маршалкин Никита

Точное решение:

$u_t= a u_{xx} + f(x, t)$

In [31]:
u_exact = lambda x, t: np.exp(-0.25 * t) * np.cos(0.5 * x) + x * (1 - x) / (10 + t)

Параметры уравнения:

In [32]:
f = lambda x, t: -x * (1 - x) / (10 + t) ** 2 + 2 / (10 + t)
a = 1

Краевые условия:

In [33]:
u_t0 = lambda x: np.cos(0.5 * x) + x * (1 - x) / 10

In [34]:
u0 = lambda t: np.exp(-0.25 * t)
u1 = lambda t: np.exp(-0.25 * t) * np.cos(0.5)

Задаем сетку по оси $x$

In [61]:
n = 100
h = (1 - 0) / n
h

0.01

In [62]:
x_net = np.linspace(0, 1, n + 1)

Находим шаг по оси $t$ исходя из условия сходимоси: $\tau \leq \frac{h^2}{2a}$

In [63]:
tau_tmp = h ** 2/ (2 * a)
m = np.int((1 - 0) / tau_tmp + 1)
tau = (1 - 0) / m
tau

4.999750012499375e-05

In [64]:
t_net = np.linspace(0, 1, m + 1)

Теоретическая погрешность

In [65]:
tau + h ** 2

0.00014999750012499374

## Явная трехточечная схема

Найдем значения на границе области

In [66]:
u = np.zeros((n + 1, m + 1))
for i in np.arange(0, n + 1):
    u[i, 0] = u_t0(x_net[i])
    
for j in np.arange(0, m + 1):
    u[0, j] = u0(t_net[j])
    u[n, j] = u1(t_net[j])

Пересчитаем значение в точке $(i, k)$ через занчения в точках $(i, k - 1)$,  $(i - 1, k - 1)$, $(i + 1, k - 1)$

In [67]:
for k in np.arange(1, m + 1):
    for i in np.arange(1, n):
        u[i, k] = (1 - 2 * a * tau / h ** 2) * u[i, k - 1] + a * tau / h ** 2 * (u[i - 1, k - 1] + u[i + 1, k - 1]) + tau * f(x_net[i], t_net[k - 1])

Посчитаем $|max(u^* - u)|$ в узлах сетки

In [68]:
max_diff = 0
for i in np.arange(0, n + 1):
    for k in np.arange(0, m + 1):
        diff = np.abs(u_exact(x_net[i], t_net[k]) - u[i, k])
        max_diff = diff if diff > max_diff else max_diff
max_diff

1.1522422982412905e-07

## Неявная трехточечная схема

In [69]:
u = np.zeros((n + 1, m + 1))
for i in np.arange(0, n + 1):
    u[i, 0] = u_t0(x_net[i])

In [70]:
A = np.identity(n + 1)
for i in np.arange(1, n):
    for j in np.arange(0, n + 1):
        if i == j + 1 or i == j - 1:
            A[i, j] = -a * tau / h ** 2
        elif i == j:
            A[i, j] = 1 + 2 * a * tau / h ** 2

In [71]:
for k in np.arange(1, m + 1):
    b = np.empty(n + 1)
    t = t_net[k]
    b[0] = u0(t)
    b[n] = u1(t)
    for i in np.arange(1, n):
        b[i] = tau * f(x_net[i], t) + u[i, k - 1]
    u[:, k] = np.linalg.solve(A, b)

Посчитаем $|max(u^* - u)|$ в узлах сетки

In [72]:
max_diff = 0
for i in np.arange(0, n + 1):
    for k in np.arange(0, m + 1):
        diff = np.abs(u_exact(x_net[i], t_net[k]) - u[i, k])
        max_diff = diff if diff > max_diff else max_diff
max_diff

2.2928024034918337e-07